In [1]:
import pandas as pd

matches = pd.read_csv("../data/raw/matches.csv", sep=";", encoding="utf-8")
squads = pd.read_csv("../data/raw/squads.csv", sep=";", encoding="utf-8")
worldcups = pd.read_csv("../data/raw/WorldCups.csv", sep=";", encoding="utf-8")
fifa_2022 = pd.read_csv("../data/raw/fifa_ranking_2022-10-06.csv", sep=",", encoding="utf-8")

players = pd.read_csv("../data/raw/players.csv", sep=",", encoding="utf-8")

ranking_soccer = pd.read_csv(
    "../data/raw/Ranking soccer 1901-2023.csv",
    sep=";",
    encoding="latin1"
)

ranking_fifa = pd.read_csv(
    "../data/raw/Ranking fifa 1993-2018.csv",
    sep=";",
    encoding="latin1"
)


/tmp/ipykernel_11340/3284431881.py:16: DtypeWarning: Columns (3,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  ranking_fifa = pd.read_csv(


Clean players.csv by keeping only the players who played in at least one worldcup since 1970

In [2]:
world_cups = [
    "1970", "1974", "1978", "1982", "1986",
    "1990", "1994", "1998", "2002", "2006",
    "2010", "2014", "2018", "2022"
]


In [3]:
players_wc = players[
    players["list_tournaments"].astype(str).apply(
        lambda x: any(wc in x for wc in world_cups)
    )
]


In [4]:
players_wc.head(10)

,key_id,player_id,family_name,given_name,birth_date,female,goal_keeper,defender,midfielder,forward,count_tournaments,list_tournaments,player_wikipedia_link
2,3,P-03484,Aaronson,Brenden,2000-10-22,0,0,0,0,1,1,2022,https://en.wikipedia.org/wiki/Brenden_Aaronson
4,5,P-03523,Abalo,Jean-Paul,1975-06-26,0,0,1,0,0,1,2006,https://en.wikipedia.org/wiki/Jean-Paul_Abalo
6,7,P-35138,Abanda,Patrice,1978-08-03,0,0,1,0,0,1,1998,https://en.wikipedia.org/wiki/Patrice_Abanda
8,9,P-75462,Abate,Ignazio,1986-11-12,0,0,1,0,0,1,2014,https://en.wikipedia.org/wiki/Ignazio_Abate
12,13,P-81393,Abbiati,Christian,1977-07-08,0,1,0,0,0,1,2002,https://en.wikipedia.org/wiki/Christian_Abbiati
13,14,P-86290,Abbondanzieri,Roberto,1972-08-19,0,1,0,0,0,1,2006,https://en.wikipedia.org/wiki/Roberto_Abbondan...
14,15,P-29132,Abd Al-Jawad,Mohamed,1962-11-28,0,0,1,0,0,1,1994,https://en.wikipedia.org/wiki/Mohamed_Abd_Al-J...
15,16,P-26446,Abdel Rahman,Adel,1967-12-11,0,0,0,0,1,1,1990,https://en.wikipedia.org/wiki/Adel_Abdel_Rahman
16,17,P-29161,Abdel Shafy,Mohamed,1985-07-01,0,0,1,0,0,1,2018,https://en.wikipedia.org/wiki/Mohamed_Abdel_Shafy
17,18,P-73679,Abdel-Hamid,Gamal,1957-11-24,0,0,0,1,0,1,1990,https://en.wikipedia.org/wiki/Gamal_Abdel-Hamid


Clean squads.csv by keeping only the squads who played from 1970 and harmonising the names

In [5]:
valid_wc = [
    "WC-1970", "WC-1974", "WC-1978", "WC-1982",
    "WC-1986", "WC-1990", "WC-1994", "WC-1998",
    "WC-2002", "WC-2006", "WC-2010", "WC-2014",
    "WC-2018", "WC-2022"
]


In [6]:
squads_wc = squads[squads["tournament_id"].isin(valid_wc)]


In [7]:
squads["team_code"] = squads["team_code"].astype(str).str.strip()
squads["team_name"] = squads["team_name"].astype(str).str.strip()

squads.loc[squads["team_code"] == "DEU", "team_name"] = "Germany"


Clean Ranking fifa 1993-2024 by keeping the rank of each squad just before the first match of the worlcup


In [8]:
matches["tournament_id"] = matches["tournament_id"].astype(str).str.strip()
matches_wc = matches[matches["tournament_id"].isin(valid_wc)].copy()

In [9]:
matches_wc["match_date_raw"] = matches_wc["match_date"].astype(str).str.strip()



In [10]:
raw = matches_wc["match_date_raw"]

d1 = pd.to_datetime(raw, format="%d.%m.%y", errors="coerce") 

matches_wc["match_date"] = d1.fillna(d1)


In [11]:
matches_wc.head()

,key_id,tournament_id,tournament_name,match_id,match_name,stage_name,group_name,group_stage,knockout_stage,replayed,...,extra_time,penalty_shootout,score_penalties,home_team_score_penalties,away_team_score_penalties,result,home_team_win,away_team_win,draw,match_date_raw
200,201,WC-1970,1970 FIFA Men's World Cup,M-1970-01,Mexico vs Soviet Union,group stage,Group 1,1,0,0,...,0,0,0-0,0,0,draw,0,0,1,31.05.70
201,202,WC-1970,1970 FIFA Men's World Cup,M-1970-02,Uruguay vs Israel,group stage,Group 2,1,0,0,...,0,0,0-0,0,0,home team win,1,0,0,02.06.70
202,203,WC-1970,1970 FIFA Men's World Cup,M-1970-03,England vs Romania,group stage,Group 3,1,0,0,...,0,0,0-0,0,0,home team win,1,0,0,02.06.70
203,204,WC-1970,1970 FIFA Men's World Cup,M-1970-04,Peru vs Bulgaria,group stage,Group 4,1,0,0,...,0,0,0-0,0,0,home team win,1,0,0,02.06.70
204,205,WC-1970,1970 FIFA Men's World Cup,M-1970-05,Belgium vs El Salvador,group stage,Group 1,1,0,0,...,0,0,0-0,0,0,home team win,1,0,0,03.06.70


In [12]:
worldcups = worldcups.copy()

worldcups["Year"] = worldcups["Year"].astype(int)

worldcups = worldcups[
    (worldcups["Year"] >= 1970) &
    (worldcups["Year"] <= 2022)
].copy()


In [13]:
worldcups["Country"] = worldcups["Country"].astype(str).str.strip()


In [14]:
name_to_code = (
    squads[["team_name", "team_code"]]
    .drop_duplicates()
    .set_index("team_name")["team_code"]
    .to_dict()
)

name_to_code.update({
    "USA": "USA",
    "United States": "USA",
    "Korea": "KOR",
    "South Korea": "KOR",
    "Korea Republic": "KOR",
    "West Germany": "DEU",
    "East Germany": "DEU",
})


In [15]:
import re

def split_hosts(s):
    parts = re.split(r"\s*/\s*|\s*&\s*|,\s*|\s+and\s+", str(s))
    return [p.strip() for p in parts if p.strip()]


In [16]:
hosts_df = (
    worldcups[["Year", "Country"]]
    .assign(host_name=lambda df: df["Country"].apply(split_hosts))
    .explode("host_name")
)

hosts_df["team_code"] = hosts_df["host_name"].map(name_to_code)

hosts_df = (
    hosts_df.rename(columns={"Year": "tournament_year"})
            [["tournament_year", "host_name", "team_code"]]
            .drop_duplicates()
            .sort_values(["tournament_year", "team_code"])
            .reset_index(drop=True)
)


In [17]:
host_codes_by_year = (
    hosts_df
    .groupby("tournament_year")["team_code"]
    .apply(lambda x: "/".join(sorted(x)))
    .reset_index()
)

worldcups_wc = worldcups.merge(
    host_codes_by_year,
    left_on="Year",
    right_on="tournament_year",
    how="left"
)

worldcups_wc = worldcups_wc.drop(columns=["tournament_year"]).rename(columns={"team_code": "host_codes"})



In [18]:
worldcups_wc.head(15)

,Year,Country,Winner,Runners-Up,Third,Fourth,GoalsScored,QualifiedTeams,MatchesPlayed,Attendance,host_codes
0,1970,Mexico,Brazil,Italy,Germany FR,Uruguay,95,16,32,1.603.975,MEX
1,1974,Germany,Germany FR,Netherlands,Poland,Brazil,97,16,38,1.865.753,DEU
2,1978,Argentina,Argentina,Netherlands,Brazil,Italy,102,16,38,1.545.791,ARG
3,1982,Spain,Italy,Germany FR,Poland,France,146,24,52,2.109.723,ESP
4,1986,Mexico,Argentina,Germany FR,France,Belgium,132,24,52,2.394.031,MEX
5,1990,Italy,Germany FR,Argentina,Italy,England,115,24,52,2.516.215,ITA
6,1994,USA,Brazil,Italy,Sweden,Bulgaria,141,24,52,3.587.538,USA
7,1998,France,France,Brazil,Croatia,Netherlands,171,32,64,2.785.100,FRA
8,2002,Korea/Japan,Brazil,Germany,Turkey,Korea Republic,161,32,64,2.705.197,JPN/KOR
9,2006,Germany,Italy,France,Germany,Portugal,147,32,64,3.359.439,DEU


In [19]:
ranking_fifa["rank_date"] = pd.to_datetime(
    ranking_fifa["rank_date"].astype(str).str.strip(),
    format="%d.%m.%y",
    errors="coerce"
)

ranking_fifa["team_code"] = ranking_fifa["country_abrv"].astype(str).str.strip()



In [20]:
ranking_fifa["rank_date"].isna().sum()
ranking_fifa[["rank_date","country_abrv"]].head()


,rank_date,country_abrv
0,1993-08-08,GER
1,1993-08-08,ITA
2,1993-08-08,SUI
3,1993-08-08,SWE
4,1993-08-08,ARG


In [21]:
matches_wc["tournament_year"] = matches_wc["tournament_id"].str.extract(r"(\d{4})").astype(int)

wc_years = [1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022]

wc_start = (
    matches_wc[matches_wc["tournament_year"].isin(wc_years)]
    .groupby("tournament_year")["match_date"]
    .min()
    .sort_index()
)
wc_start


tournament_year
1970   1970-05-31
1974   1974-06-13
1978   1978-06-01
1982   1982-06-13
1986   1986-05-31
1990   1990-06-08
1994   1994-06-17
1998   1998-06-10
2002   2002-05-31
2006   2006-06-09
2010   2010-06-11
2014   2014-06-12
2018   2018-06-14
2022   2022-11-20
Name: match_date, dtype: datetime64[ns]

In [22]:
pre_wc_list = []

for year in [1994, 1998, 2002, 2006, 2010, 2014, 2018]:
    cutoff = wc_start.loc[year]
    
    tmp = ranking_fifa[ranking_fifa["rank_date"] < cutoff].copy()
    tmp = tmp.sort_values("rank_date").groupby("team_code", as_index=False).tail(1)
    
    tmp["tournament_year"] = year
    tmp["wc_start_date"] = cutoff
    
    pre_wc_list.append(tmp)

ranking_fifa_pre_wc = pd.concat(pre_wc_list, ignore_index=True)


In [23]:
ranking_fifa_clean = ranking_fifa_pre_wc[[
    "tournament_year",
    "team_code",
    "rank_date",
    "rank",
    "total_points",
    "wc_start_date"
]].copy()


In [24]:
code_fix_2022 = {
    "GER": "DEU",
    "SUI": "CHE",
    "URU": "URY",
    "CRO": "HRV",
    "CRC": "CRI",
    "DEN": "DNK",
    "POR": "PRT",
    "NED": "NLD",
    "KSA": "SAU",
}

fifa_2022["team_code"] = (
    fifa_2022["team_code"]
    .astype(str)
    .str.strip()
    .replace(code_fix_2022)
)

# optionnel mais propre : strip team
fifa_2022["team"] = fifa_2022["team"].astype(str).str.strip()


In [25]:
fifa_2022_clean = fifa_2022.copy()

fifa_2022_clean["tournament_year"] = 2022
fifa_2022_clean["rank_date"] = pd.to_datetime("2022-10-06")

fifa_2022_clean = fifa_2022_clean.rename(columns={"points": "total_points"})


In [26]:
teams_2022 = (
    squads[squads["tournament_id"] == "WC-2022"][["team_code"]]
    .drop_duplicates()
)

fifa_2022_clean = teams_2022.merge(
    fifa_2022_clean,
    on="team_code",
    how="left"
)


In [27]:
fifa_2022_clean["wc_start_date"] = wc_start.loc[2022]


In [28]:
fifa_2022_clean = fifa_2022_clean[[
    "tournament_year",
    "team_code",
    "rank_date",
    "rank",
    "total_points",
    "wc_start_date"
]].copy()


In [29]:
ranking_fifa_clean = ranking_fifa_clean[ranking_fifa_clean["tournament_year"] != 2022].copy()

ranking_fifa_clean = pd.concat(
    [ranking_fifa_clean, fifa_2022_clean],
    ignore_index=True
)


In [30]:
ranking_fifa_clean.columns


Index(['tournament_year', 'team_code', 'rank_date', 'rank', 'total_points',
       'wc_start_date'],
      dtype='object')

In [31]:
soccer_clean = ranking_soccer[["year", "rank", "team"]].copy()

soccer_clean["year"] = soccer_clean["year"].astype(int)
soccer_clean["rank"] = pd.to_numeric(soccer_clean["rank"], errors="coerce")
soccer_clean["team"] = soccer_clean["team"].astype(str).str.strip()


In [32]:
wc_years_soccer = [1970, 1974, 1978, 1982, 1986, 1990]
soccer_clean = soccer_clean[soccer_clean["year"].isin(wc_years_soccer)].copy()


In [33]:
fifa_small = ranking_fifa_clean[["tournament_year", "team_code", "rank"]].copy()


In [34]:

name_to_code = (
    squads[["team_name", "team_code"]]
    .drop_duplicates()
    .set_index("team_name")["team_code"]
    .to_dict()
)


name_to_code.update({
    "West Germany": "DEU",
    "Germany FR": "DEU",
    "Soviet Union": "SUN",
    "Yugoslavia": "YUG",
    "Czechoslovakia": "CSK",
    "Republic of Ireland": "IRL",
    "United States": "USA",
    "South Korea": "KOR",
    "North Korea": "PRK",
    "Zaire": "COD",
})

soccer_small = soccer_clean.copy()
soccer_small["team_code"] = soccer_small["team"].map(name_to_code)
soccer_small = soccer_small.rename(columns={"year": "tournament_year"})

soccer_small = soccer_small[["tournament_year", "team_code", "rank"]].copy()


In [35]:
ranking_all = pd.concat([soccer_small, fifa_small], ignore_index=True)

In [36]:
soccer_clean[soccer_small["team_code"].isna()][["year","team"]].drop_duplicates()


,year,team
5622,1970,DR Congo
5627,1970,Sudan
5639,1970,Guinea
5640,1970,Guatemala
5647,1970,Congo
...,...,...
9540,1990,Western Samoa
9541,1990,Kiribati
9542,1990,Anguilla
9543,1990,Eastern Samoa


In [37]:
teams_wc_7090 = (
    squads[squads["tournament_id"].isin([
        "WC-1970","WC-1974","WC-1978","WC-1982","WC-1986","WC-1990"
    ])][["team_code"]]
    .drop_duplicates()
)

In [38]:
soccer_small = soccer_clean.copy()
soccer_small["team_code"] = soccer_small["team"].map(name_to_code)

In [39]:
soccer_small = soccer_small[soccer_small["team_code"].isin(teams_wc_7090["team_code"])]

In [40]:
soccer_small = soccer_small.rename(columns={"year": "tournament_year"})

In [41]:
soccer_small = soccer_small[["tournament_year","team_code","rank"]].copy()

In [42]:
ranking_all = pd.concat([soccer_small, fifa_small], ignore_index=True)


In [43]:
fifa_small.columns

Index(['tournament_year', 'team_code', 'rank'], dtype='object')

In [44]:
squads_wc = squads_wc.copy()
squads_wc["tournament_year"] = squads_wc["tournament_id"].str.extract(r"(\d{4})").astype(int)


In [45]:
matches_wc["tournament_year"] = matches_wc["tournament_id"].str.extract(r"(\d{4})").astype(int)

wc_start = (
    matches_wc.groupby("tournament_year")["match_date"]
    .min()
    .to_dict()
)

In [46]:
players_wc = players_wc.copy()
players_wc["birth_date"] = pd.to_datetime(players_wc["birth_date"], errors="coerce")


In [47]:
squad_players = squads_wc[["tournament_year","team_code","player_id"]].drop_duplicates()

In [48]:
squad_players = squad_players.merge(
    players_wc[["player_id","birth_date"]],
    on="player_id",
    how="left"
)

In [49]:
squad_players["tournament_start"] = squad_players["tournament_year"].map(wc_start)
squad_players["tournament_start"] = pd.to_datetime(squad_players["tournament_start"], errors="coerce")


In [50]:
squad_players["age"] = (squad_players["tournament_start"] - squad_players["birth_date"]).dt.days / 365.25

avg_age = (
    squad_players.groupby(["tournament_year","team_code"], as_index=False)["age"]
    .mean()
    .rename(columns={"age":"avg_team_age"})
)

avg_age.tail()

,tournament_year,team_code,avg_team_age
363,2022,SRB,27.361133
364,2022,TUN,28.537145
365,2022,URY,28.309167
366,2022,USA,25.585847
367,2022,WAL,26.871900


In [51]:
hosts = worldcups_wc[["Year", "host_codes"]].rename(
    columns={"Year": "tournament_year"}
).copy()

hosts = hosts.assign(
    team_code=hosts["host_codes"].astype(str).str.split("/")
).explode("team_code")

hosts["team_code"] = hosts["team_code"].str.strip()
hosts["is_host"] = 1

hosts = hosts[["tournament_year", "team_code", "is_host"]].drop_duplicates()


In [52]:
teams_wc = (
    squads_wc[["tournament_year", "team_code"]]
    .drop_duplicates()
    .sort_values(["tournament_year","team_code"])
    .reset_index(drop=True)
)



In [53]:
teams_wc = teams_wc.merge(
    avg_age,
    on=["tournament_year","team_code"],
    how="left"
)


In [54]:
hosts = worldcups_wc[["Year", "host_codes"]].rename(columns={"Year":"tournament_year"}).copy()
hosts = hosts.assign(team_code=hosts["host_codes"].astype(str).str.split("/")).explode("team_code")
hosts["team_code"] = hosts["team_code"].str.strip()
hosts["is_host"] = 1
hosts = hosts[["tournament_year","team_code","is_host"]].drop_duplicates()

teams_wc = teams_wc.merge(hosts, on=["tournament_year","team_code"], how="left")
teams_wc["is_host"] = teams_wc["is_host"].fillna(0).astype(int)


In [55]:
strength = ranking_all.copy()
strength["rank"] = pd.to_numeric(strength["rank"], errors="coerce")
strength = strength.dropna(subset=["rank"])
strength["strength_pre"] = (1/strength["rank"])
strength = strength[["tournament_year","team_code","strength_pre"]].drop_duplicates()

teams_wc = teams_wc.merge(strength, on=["tournament_year","team_code"], how="left")


In [56]:
matches_wc["home_team_score"] = pd.to_numeric(matches_wc["home_team_score"], errors="coerce")
matches_wc["away_team_score"] = pd.to_numeric(matches_wc["away_team_score"], errors="coerce")

In [57]:
home = matches_wc[["tournament_year","group_stage","home_team_code","home_team_score","away_team_score"]].rename(
    columns={"home_team_code":"team_code","home_team_score":"gf","away_team_score":"ga"}
)
away = matches_wc[["tournament_year","group_stage","away_team_code","away_team_score","home_team_score"]].rename(
    columns={"away_team_code":"team_code","away_team_score":"gf","home_team_score":"ga"}
)

m_long = pd.concat([home, away], ignore_index=True)

In [58]:
m_long["win"] = (m_long["gf"] > m_long["ga"]).astype(int)
m_long["draw"] = (m_long["gf"] == m_long["ga"]).astype(int)
m_long["loss"] = (m_long["gf"] < m_long["ga"]).astype(int)
m_long["clean_sheet"] = (m_long["ga"] == 0).astype(int)

In [59]:
group_stats = (
    m_long[m_long["group_stage"] == 1]
    .groupby(["tournament_year","team_code"], as_index=False)
    .agg(
        matches_group=("group_stage","size"),
        wins=("win","sum"),
        draws=("draw","sum"),
        losses=("loss","sum"),
        goals_for=("gf","sum"),
        goals_against=("ga","sum"),
        clean_sheet=("clean_sheet","sum"),
    )
)

group_stats["points"] = 3*group_stats["wins"] + group_stats["draws"]
group_stats["goal_diff"] = group_stats["goals_for"] - group_stats["goals_against"]
group_stats["avg_goals_for"] = group_stats["goals_for"] / group_stats["matches_group"]
group_stats["avg_goals_against"] = group_stats["goals_against"] / group_stats["matches_group"]
group_stats["win_ratio"] = group_stats["wins"] / group_stats["matches_group"]

In [60]:
final = teams_wc.merge(group_stats, on=["tournament_year","team_code"], how="left")

In [61]:
stage_to_yord = {
    "group stage": 0,
    "second group stage": 2,
    "round of 16": 1,
    "quarter-finals": 2,
    "semi-finals": 3,
    "third-place match": 3,
    "final": 4,
}

In [62]:
home = matches_wc[["tournament_year","stage_name","home_team_code"]].rename(columns={"home_team_code":"team_code"})
away = matches_wc[["tournament_year","stage_name","away_team_code"]].rename(columns={"away_team_code":"team_code"})
stages_long = pd.concat([home, away], ignore_index=True)

stages_long["stage_name"] = stages_long["stage_name"].str.strip().str.lower()
stages_long["y_ord_tmp"] = stages_long["stage_name"].map(stage_to_yord)

y_from_matches = (
    stages_long
    .groupby(["tournament_year","team_code"], as_index=False)["y_ord_tmp"]
    .max()
    .rename(columns={"y_ord_tmp":"y_ord"})
)

In [63]:
name_to_code = (
    squads_wc[["team_name","team_code"]]
    .drop_duplicates()
    .set_index("team_name")["team_code"]
    .to_dict()
)
name_to_code.update({"Germany FR":"DEU", "West Germany":"DEU"})

top4 = worldcups_wc[["Year","Winner","Runners-Up","Third","Fourth"]].rename(columns={"Year":"tournament_year"}).copy()

top4_long = top4.melt(
    id_vars="tournament_year",
    value_vars=["Winner","Runners-Up","Third","Fourth"],
    var_name="place",
    value_name="team_name"
)

top4_long["team_code"] = top4_long["team_name"].astype(str).str.strip().map(name_to_code)

place_to_yord = {
    "Winner": 5,
    "Runners-Up": 4,
    "Third": 3,
    "Fourth": 3,
}

top4_long["y_ord"] = top4_long["place"].map(place_to_yord)

y_top4 = top4_long[["tournament_year","team_code","y_ord"]].dropna().drop_duplicates()


In [64]:
y_ord_df = pd.concat([y_from_matches, y_top4], ignore_index=True)
y_ord_df = y_ord_df.groupby(["tournament_year","team_code"], as_index=False)["y_ord"].max()

final = final.drop(columns=["y_ord","y_label"], errors="ignore")
final = final.merge(y_ord_df, on=["tournament_year","team_code"], how="left")

final["y_ord"] = final["y_ord"].fillna(0).astype(int)

In [65]:
LABELS = ["Group", "R16", "QF", "SF", "Final", "Winner"]
final["y_label"] = final["y_ord"].map({i: lab for i, lab in enumerate(LABELS)})


In [66]:
final["split"] = final["tournament_year"].apply(lambda y: "train" if y <= 2014 else "test")


In [67]:
final.head()

,tournament_year,team_code,avg_team_age,is_host,strength_pre,matches_group,wins,draws,losses,goals_for,goals_against,clean_sheet,points,goal_diff,avg_goals_for,avg_goals_against,win_ratio,y_ord,y_label,split
0,1970,BEL,26.261963,0,0.041667,3,1,0,2,4,5,1,3,-1,1.333333,1.666667,0.333333,0,Group,train
1,1970,BGR,26.315848,0,0.035714,3,0,1,2,5,9,0,1,-4,1.666667,3.000000,0.000000,0,Group,train
2,1970,BRA,24.965715,0,1.000000,3,3,0,0,8,3,1,9,5,2.666667,1.000000,1.000000,5,Winner,train
3,1970,CSK,25.473213,0,0.071429,3,0,0,3,2,7,0,0,-5,0.666667,2.333333,0.000000,0,Group,train
4,1970,DEU,27.727335,0,0.333333,3,3,0,0,10,4,0,9,6,3.333333,1.333333,1.000000,3,SF,train


In [68]:
final.to_csv(
    "../data/clean/final_dataset1.csv",
    index=False
)

In [69]:
train_df[FEATURES_ENRICHED + ["y_ord"]].corr(numeric_only=True)["y_ord"].sort_values(ascending=False)

NameError: name 'train_df' is not defined